<a href="https://colab.research.google.com/github/Ayy4s/Develhope-Exercises/blob/main/PySpark_and_SparkSQL_Complete_Guide.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PySpark and SparkSQL Complete Guides

Apache Spark is a cluster computing system that offers comprehensive libraries and APIs for developers, and
SparkSQL can be represented as the module in Apache Spark for processing unstructured data with the help of DataFrame API.

In this notebook, we will cover the basics how to run Spark Jobs with PySpark (Python API) and execute useful functions insdie. If followed, you should be able to grasp a basic understadning of PySparks and its common functions.

In [ ]:
#we use the findspark library to locate spark on our local machine
import findspark
findspark.init()

In [ ]:
import pandas as pd
import numpy as np
from datetime import date, timedelta, datetime
import time

import pyspark # only run this after findspark.init()
from pyspark.sql import SparkSession, SQLContext
from pyspark.context import SparkContext
from pyspark.sql.functions import *
from pyspark.sql.types import *

### 1. Initialize the Spark Session

We need to begin with initilize the Spark Session. DataFrame can be created and registered as tables. Moreover, SQL tables be executed, tables can be cached, and parquet/json/csv/avro data formatted files can be read.

In [ ]:
sc = SparkSession.builder.appName("PysparkExample").config ("spark.sql.shuffle.partitions", "50").config("spark.driver.maxResultSize","5g").config ("spark.sql.execution.arrow.enabled", "true").getOrCreate()

In [ ]:
sc

### 2. Load Data

You can download the Kaggle dataset, which includes the book title, author, the date of the best seller list, the published date of the list, the book description, the rank (this week and last week), the publisher, number of weeks on the list, and the price [Link](https://www.kaggle.com/cmenca/new-york-times-hardcover-fiction-best-sellers).

Spark is so Awesome that it supports all different types of data to be read.

DataFrames can be created by reading txt, csv, json and parquet file formats. In our example, we will be using .json formatted file. You can also find and read text, csv and parquet file formats by using the related read functions as shown below.

In [ ]:
#JSON
dataframe = sc.read.json('./data/nyt2.json')

#TXT files
# dataframe_txt = sc.read.text('./data/text_data.txt')

#CSV files
# dataframe_csv = sc.read.csv('./data/csv_data.csv')

#PARQUET files
# dataframe_parquet = sc.read.load('./data/parquet_data.parquet')

#### Look at the data with show()

In [ ]:
dataframe.show(10)

+--------------------+--------------------+--------------------+-----------------+--------------------+--------+-----------------+-------------+----+--------------+--------------------+-------------+
|                 _id|  amazon_product_url|              author| bestsellers_date|         description|   price|   published_date|    publisher|rank|rank_last_week|               title|weeks_on_list|
+--------------------+--------------------+--------------------+-----------------+--------------------+--------+-----------------+-------------+----+--------------+--------------------+-------------+
|[5b4aa4ead3089013...|http://www.amazon...|       Dean R Koontz|[[1211587200000]]|Odd Thomas, who c...|  [, 27]|[[1212883200000]]|       Bantam| [1]|           [0]|           ODD HOURS|          [1]|
|[5b4aa4ead3089013...|http://www.amazon...|     Stephenie Meyer|[[1211587200000]]|Aliens have taken...|[25.99,]|[[1212883200000]]|Little, Brown| [2]|           [1]|            THE HOST|          [3]|


#### Simple Data Inspection

We usually want to simply skim through the dataframe before going deeper

In [ ]:
# Returns dataframe column names and data types
dataframe.dtypes

# Displays the content of dataframe
dataframe.show()

# Return first n rows
dataframe.head()

# Returns first row
dataframe.first()

# Return first n rows
dataframe.take(5)

# Computes summary statistics
dataframe.describe().show()

# Returns columns of dataframe
dataframe.columns

# Counts the number of rows in dataframe
dataframe.count()

# Counts the number of distinct rows in dataframe
dataframe.distinct().count()

# Prints plans including physical and logical
dataframe.explain(4)

+--------------------+--------------------+--------------------+-----------------+--------------------+--------+-----------------+--------------------+----+--------------+--------------------+-------------+
|                 _id|  amazon_product_url|              author| bestsellers_date|         description|   price|   published_date|           publisher|rank|rank_last_week|               title|weeks_on_list|
+--------------------+--------------------+--------------------+-----------------+--------------------+--------+-----------------+--------------------+----+--------------+--------------------+-------------+
|[5b4aa4ead3089013...|http://www.amazon...|       Dean R Koontz|[[1211587200000]]|Odd Thomas, who c...|  [, 27]|[[1212883200000]]|              Bantam| [1]|           [0]|           ODD HOURS|          [1]|
|[5b4aa4ead3089013...|http://www.amazon...|     Stephenie Meyer|[[1211587200000]]|Aliens have taken...|[25.99,]|[[1212883200000]]|       Little, Brown| [2]|           [1]| 

### 3. Useful common functions

#### [1] Remove Duplicate Values

Duplicate values in a table can be eliminated by using dropDuplicates() function.

In [ ]:
dataframe_dropdup = dataframe.dropDuplicates()
dataframe_dropdup.show(10)

+--------------------+--------------------+--------------------+-----------------+--------------------+--------+-----------------+----------------+----+--------------+--------------------+-------------+
|                 _id|  amazon_product_url|              author| bestsellers_date|         description|   price|   published_date|       publisher|rank|rank_last_week|               title|weeks_on_list|
+--------------------+--------------------+--------------------+-----------------+--------------------+--------+-----------------+----------------+----+--------------+--------------------+-------------+
|[5b4aa4ead3089013...|http://www.amazon...|Clive Cussler wit...|[[1213401600000]]|Juan Cabrillo and...|[26.95,]|[[1214697600000]]|          Putnam| [4]|           [3]|         PLAGUE SHIP|          [2]|
|[5b4aa4ead3089013...|http://www.amazon...|      Jeffery Deaver|[[1215820800000]]|Detectives Lincol...|   [, 0]|[[1217116800000]]|Simon & Schuster|[20]|           [0]|   THE BROKEN WINDOW|

#### [2] 'Select' Operation

It is possible to obtain columns by column or by indexing (i.e. dataframe[‘author’]).

In [ ]:
#Show all entries in title column
dataframe.select("author").show(10)

#Show all entries in title, author, rank, price columns
dataframe.select("author", "title", "rank", "price").show(10)

+--------------------+
|              author|
+--------------------+
|       Dean R Koontz|
|     Stephenie Meyer|
|        Emily Giffin|
|   Patricia Cornwell|
|     Chuck Palahniuk|
|James Patterson a...|
|       John Sandford|
|       Jimmy Buffett|
|    Elizabeth George|
|      David Baldacci|
+--------------------+
only showing top 10 rows

+--------------------+--------------------+----+--------+
|              author|               title|rank|   price|
+--------------------+--------------------+----+--------+
|       Dean R Koontz|           ODD HOURS| [1]|  [, 27]|
|     Stephenie Meyer|            THE HOST| [2]|[25.99,]|
|        Emily Giffin|LOVE THE ONE YOU'...| [3]|[24.95,]|
|   Patricia Cornwell|           THE FRONT| [4]|[22.95,]|
|     Chuck Palahniuk|               SNUFF| [5]|[24.95,]|
|James Patterson a...|SUNDAYS AT TIFFANY’S| [6]|[24.99,]|
|       John Sandford|        PHANTOM PREY| [7]|[26.95,]|
|       Jimmy Buffett|          SWINE NOT?| [8]|[21.99,]|
|    Elizabeth

#### [3] 'When' Operation

In [ ]:
# Show title and assign 0 or 1 depending on title
dataframe.select("title", when(dataframe.title != 'ODD HOURS', 1).otherwise(0)).show(10)

+--------------------+-----------------------------------------------------+
|               title|CASE WHEN (NOT (title = ODD HOURS)) THEN 1 ELSE 0 END|
+--------------------+-----------------------------------------------------+
|           ODD HOURS|                                                    0|
|            THE HOST|                                                    1|
|LOVE THE ONE YOU'...|                                                    1|
|           THE FRONT|                                                    1|
|               SNUFF|                                                    1|
|SUNDAYS AT TIFFANY’S|                                                    1|
|        PHANTOM PREY|                                                    1|
|          SWINE NOT?|                                                    1|
|     CARELESS IN RED|                                                    1|
|     THE WHOLE TRUTH|                                                    1|

#### [4] 'isin' Operation

In [ ]:
# Show rows with specified authors if in the given options
dataframe [dataframe.author.isin("John Sandford", "Emily Giffin")].show(5)

+--------------------+--------------------+-------------+-----------------+--------------------+--------+-----------------+------------+----+--------------+--------------------+-------------+
|                 _id|  amazon_product_url|       author| bestsellers_date|         description|   price|   published_date|   publisher|rank|rank_last_week|               title|weeks_on_list|
+--------------------+--------------------+-------------+-----------------+--------------------+--------+-----------------+------------+----+--------------+--------------------+-------------+
|[5b4aa4ead3089013...|http://www.amazon...| Emily Giffin|[[1211587200000]]|A woman's happy m...|[24.95,]|[[1212883200000]]|St. Martin's| [3]|           [2]|LOVE THE ONE YOU'...|          [2]|
|[5b4aa4ead3089013...|http://www.amazon...|John Sandford|[[1211587200000]]|The Minneapolis d...|[26.95,]|[[1212883200000]]|      Putnam| [7]|           [4]|        PHANTOM PREY|          [3]|
|[5b4aa4ead3089013...|http://www.amazon.

#### [5] 'Like' Operation

In [ ]:
# Show author and title is TRUE if title has " THE " word in titles
dataframe.select("author", "title", dataframe.title.like("% THE %")).show(15)

+--------------------+--------------------+------------------+
|              author|               title|title LIKE % THE %|
+--------------------+--------------------+------------------+
|       Dean R Koontz|           ODD HOURS|             false|
|     Stephenie Meyer|            THE HOST|             false|
|        Emily Giffin|LOVE THE ONE YOU'...|              true|
|   Patricia Cornwell|           THE FRONT|             false|
|     Chuck Palahniuk|               SNUFF|             false|
|James Patterson a...|SUNDAYS AT TIFFANY’S|             false|
|       John Sandford|        PHANTOM PREY|             false|
|       Jimmy Buffett|          SWINE NOT?|             false|
|    Elizabeth George|     CARELESS IN RED|             false|
|      David Baldacci|     THE WHOLE TRUTH|             false|
|        Troy Denning|          INVINCIBLE|             false|
|          James Frey|BRIGHT SHINY MORNING|             false|
|         Garth Stein|THE ART OF RACING...|            

#### [6] 'Startswith' — 'Endswith' Operation

StartsWith scans from the beginning of word/content with specified criteria in the brackets. In parallel, EndsWith processes the word/content starting from the end. Both of the functions are case sensitive.

In [ ]:
dataframe.select("author", "title", dataframe.title.startswith("THE")).show(5)
dataframe.select("author", "title", dataframe.title.endswith("NT")).show(5)

+-----------------+--------------------+----------------------+
|           author|               title|startswith(title, THE)|
+-----------------+--------------------+----------------------+
|    Dean R Koontz|           ODD HOURS|                 false|
|  Stephenie Meyer|            THE HOST|                  true|
|     Emily Giffin|LOVE THE ONE YOU'...|                 false|
|Patricia Cornwell|           THE FRONT|                  true|
|  Chuck Palahniuk|               SNUFF|                 false|
+-----------------+--------------------+----------------------+
only showing top 5 rows

+-----------------+--------------------+-------------------+
|           author|               title|endswith(title, NT)|
+-----------------+--------------------+-------------------+
|    Dean R Koontz|           ODD HOURS|              false|
|  Stephenie Meyer|            THE HOST|              false|
|     Emily Giffin|LOVE THE ONE YOU'...|              false|
|Patricia Cornwell|           THE

#### [7] 'Substring' Operation

In the following examples, texts are extracted from the index numbers (1, 3), (3, 6) and (1, 6).

In [ ]:
dataframe.select(dataframe.author.substr(1, 3).alias("title")).show(5)
dataframe.select(dataframe.author.substr(3, 6).alias("title")).show(5)
dataframe.select(dataframe.author.substr(1, 6).alias("title")).show(5)

+-----+
|title|
+-----+
|  Dea|
|  Ste|
|  Emi|
|  Pat|
|  Chu|
+-----+
only showing top 5 rows

+------+
| title|
+------+
|an R K|
|epheni|
|ily Gi|
|tricia|
|uck Pa|
+------+
only showing top 5 rows

+------+
| title|
+------+
|Dean R|
|Stephe|
|Emily |
|Patric|
|Chuck |
+------+
only showing top 5 rows



#### [8] Adding Columns

In [ ]:
from pyspark.sql import functions as F

# Lit() is required while we are creating columns with exact values.
dataframe = dataframe.withColumn('new_column', F.lit('This is a new column'))

display(dataframe)

DataFrame[_id: struct<$oid:string>, amazon_product_url: string, author: string, bestsellers_date: struct<$date:struct<$numberLong:string>>, description: string, price: struct<$numberDouble:string,$numberInt:string>, published_date: struct<$date:struct<$numberLong:string>>, publisher: string, rank: struct<$numberInt:string>, rank_last_week: struct<$numberInt:string>, title: string, weeks_on_list: struct<$numberInt:string>, new_column: string]

#### [9] Updating Columns

For updated operations of DataFrame API, withColumnRenamed() function is used with two parameters.

In [ ]:
# Update column 'amazon_product_url' with 'URL'
dataframe = dataframe.withColumnRenamed('amazon_product_url', 'URL')

dataframe.show(5)

+--------------------+--------------------+-----------------+-----------------+--------------------+--------+-----------------+-------------+----+--------------+--------------------+-------------+--------------------+
|                 _id|                 URL|           author| bestsellers_date|         description|   price|   published_date|    publisher|rank|rank_last_week|               title|weeks_on_list|          new_column|
+--------------------+--------------------+-----------------+-----------------+--------------------+--------+-----------------+-------------+----+--------------+--------------------+-------------+--------------------+
|[5b4aa4ead3089013...|http://www.amazon...|    Dean R Koontz|[[1211587200000]]|Odd Thomas, who c...|  [, 27]|[[1212883200000]]|       Bantam| [1]|           [0]|           ODD HOURS|          [1]|This is a new column|
|[5b4aa4ead3089013...|http://www.amazon...|  Stephenie Meyer|[[1211587200000]]|Aliens have taken...|[25.99,]|[[1212883200000]]|L

#### [10] Removing Columns

Removal of a column can be achieved in two ways: \
1. Adding the list of column names in the drop() function
2. Specifying columns by pointing in the drop function

In [ ]:
# 1.
dataframe_remove = dataframe.drop("publisher", "published_date").show(5)

# 2.
dataframe_remove2 = dataframe.drop(dataframe.publisher).drop(dataframe.published_date).show(5)

+--------------------+--------------------+-----------------+-----------------+--------------------+--------+----+--------------+--------------------+-------------+--------------------+
|                 _id|                 URL|           author| bestsellers_date|         description|   price|rank|rank_last_week|               title|weeks_on_list|          new_column|
+--------------------+--------------------+-----------------+-----------------+--------------------+--------+----+--------------+--------------------+-------------+--------------------+
|[5b4aa4ead3089013...|http://www.amazon...|    Dean R Koontz|[[1211587200000]]|Odd Thomas, who c...|  [, 27]| [1]|           [0]|           ODD HOURS|          [1]|This is a new column|
|[5b4aa4ead3089013...|http://www.amazon...|  Stephenie Meyer|[[1211587200000]]|Aliens have taken...|[25.99,]| [2]|           [1]|            THE HOST|          [3]|This is a new column|
|[5b4aa4ead3089013...|http://www.amazon...|     Emily Giffin|[[1211587

#### [11] 'GroupBy' Operation

In [ ]:
# Group by author, count the books of the authors in the groups

dataframe.groupBy("author").count().show(10)

+--------------------+-----+
|              author|count|
+--------------------+-----+
|          James Frey|    2|
|    Elin Hilderbrand|   58|
|   Sharon Kay Penman|    2|
|         Kate Jacobs|    3|
|       Karen Robards|    6|
|     Gary Shteyngart|    3|
|         Lisa Genova|    7|
|James Patterson a...|   30|
|         Ruth Reichl|    3|
|         JRR Tolkien|    2|
+--------------------+-----+
only showing top 10 rows



#### [12] 'Filter' Operation

In [ ]:
# Filtering entries of title
# Only keeps records having value 'THE HOST'

dataframe.filter(dataframe["title"] == 'THE HOST').show(5)

+--------------------+--------------------+---------------+-----------------+--------------------+--------+-----------------+-------------+----+--------------+--------+-------------+--------------------+
|                 _id|                 URL|         author| bestsellers_date|         description|   price|   published_date|    publisher|rank|rank_last_week|   title|weeks_on_list|          new_column|
+--------------------+--------------------+---------------+-----------------+--------------------+--------+-----------------+-------------+----+--------------+--------+-------------+--------------------+
|[5b4aa4ead3089013...|http://www.amazon...|Stephenie Meyer|[[1211587200000]]|Aliens have taken...|[25.99,]|[[1212883200000]]|Little, Brown| [2]|           [1]|THE HOST|          [3]|This is a new column|
|[5b4aa4ead3089013...|http://www.amazon...|Stephenie Meyer|[[1212192000000]]|Aliens have taken...|[25.99,]|[[1213488000000]]|Little, Brown| [2]|           [2]|THE HOST|          [4]|Th

#### [13] Handling Missing Values

In [ ]:
# Replacing null values
dataframe.na.fill()
dataFrame.fillna()
dataFrameNaFunctions.fill()

# Returning new dataframe restricting rows with null valuesdataframe.na.drop()
dataFrame.dropna()
dataFrameNaFunctions.drop()

# Return new dataframe replacing one value with another
dataframe.na.replace(5, 15)
dataFrame.replace()
dataFrameNaFunctions.replace()

#### [14] Repartitioning

It is possible to increase or decrease the existing level of partitioning in RDD. \

Increasing can be actualized by using **repartition(self, numPartitions)** function which results in a new RDD that obtains same/higher number of partitions. \

Decreasing can be processed with **coalesce(self, numPartitions, shuffle=False)** function that results in new RDD with a reduced number of partitions to a specified number

In [ ]:
# Dataframe with 10 partitions
dataframe.repartition(10).rdd.getNumPartitions()

# Dataframe with 1 partition
dataframe.coalesce(1).rdd.getNumPartitions()

#### [15] Running SQL Commnads In Spark

In [ ]:
# Registering a table
dataframe.registerTempTable("df")

sc.sql("select * from df").show(3)

+--------------------+--------------------+---------------+-----------------+--------------------+--------+-----------------+-------------+----+--------------+--------------------+-------------+--------------------+
|                 _id|                 URL|         author| bestsellers_date|         description|   price|   published_date|    publisher|rank|rank_last_week|               title|weeks_on_list|          new_column|
+--------------------+--------------------+---------------+-----------------+--------------------+--------+-----------------+-------------+----+--------------+--------------------+-------------+--------------------+
|[5b4aa4ead3089013...|http://www.amazon...|  Dean R Koontz|[[1211587200000]]|Odd Thomas, who c...|  [, 27]|[[1212883200000]]|       Bantam| [1]|           [0]|           ODD HOURS|          [1]|This is a new column|
|[5b4aa4ead3089013...|http://www.amazon...|Stephenie Meyer|[[1211587200000]]|Aliens have taken...|[25.99,]|[[1212883200000]]|Little, Bro

In [ ]:
sc.sql("SELECT CASE WHEN description LIKE '%love%' THEN 'Love_Theme'
           WHEN description LIKE '%hate%' THEN 'Hate_Theme'
           WHEN description LIKE '%happy%' THEN 'Happiness_Theme' \
           WHEN description LIKE '%anger%' THEN 'Anger_Theme' \
           WHEN description LIKE '%horror%' THEN 'Horror_Theme' \
           WHEN description LIKE '%death%' THEN 'Criminal_Theme' \
           WHEN description LIKE '%detective%' THEN 'Mystery_Theme' ELSE 'Other_Themes' END Themes from df").groupBy('Themes').count().show()

#### [16] Output in Data

In [ ]:
# Converting dataframe into an RDD
rdd_convert = dataframe.rdd

# Converting dataframe into a RDD of string
dataframe.toJSON().first()

# Obtaining contents of df as Pandas
dataFramedataframe.toPandas()

#### [17] Write & Save to Files

Any data source type that is loaded to our code as data frames can easily be converted and saved into other types including .parquet and .json

In [ ]:
# Write & Save File in .parquet format
dataframe.select("author", "title", "rank", "description") \
        .write \
        .save("Rankings_Descriptions.parquet")

In [ ]:
# Write & Save File in .json format
dataframe.select("author", "title") \
        .write \
        .save("Authors_Titles.json",format="json")

#### [18] End Spark Session

In [ ]:
sc.stop()